In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] ='/content'
!kaggle datasets download -d uciml/breast-cancer-wisconsin-data
!unzip \*.zip && rm *.zip

  0% 0.00/48.6k [00:00<?, ?B/s]
100% 48.6k/48.6k [00:00<00:00, 1.65MB/s]
Archive:  breast-cancer-wisconsin-data.zip
  inflating: data.csv                


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

# Define multiple kernel function
def gaussian_kernel(x, y, sigma):
    return np.exp(-np.linalg.norm(x - y) ** 2 / (2 * (sigma ** 2)))

# Implement MKNN algorithm
def mknn(X_train, y_train, X_test, k, sigmas):
    y_pred = []
    for x_test in X_test:
        votes = {}
        for sigma in sigmas:
            distances = [gaussian_kernel(x_test, x_train, sigma) for x_train in X_train]
            nearest_indices = np.argsort(distances)[:k]
            nearest_labels = [y_train[i] for i in nearest_indices]
            for label in nearest_labels:
                if label in votes:
                    votes[label] += 1
                else:
                    votes[label] = 1
        pred_label = max(votes, key=votes.get)
        y_pred.append(pred_label)
    return y_pred

# Step 1: Load Data
data = pd.read_csv('/content/data.csv')

# Assuming 'diagnosis' is the target variable
X = data[['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
          'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave points_mean']]
y = data['diagnosis']  # Target variable

# Step 2: Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 4: Define the number of neighbors (k) and sigmas for multiple kernels
k = 5  # Number of neighbors
sigmas = [0.1, 0.5, 1.0]  # List of sigmas

# Step 5: Apply MKNN algorithm
y_pred = mknn(X_train_scaled, y_train, X_test_scaled, k, sigmas)

# Step 6: Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

print("Classification Report:\n", classification_report(y_test, y_pred))


KeyError: 222